# TD 3 : COINTEGRATION

![CentraleSupelec Logo](https://www.centralesupelec.fr/sites/all/themes/cs_theme/medias/common/images/intro/logo_nouveau.jpg)

## But du TD : 
* Load the “logret_russel1000_pairtrad.csv” and obtain log-prices
* Split the date in in-sample (252 days) and out-of-sample (252 days). 
* Select the subset of stocks that are I(1) in the in-sample.
* Select the pair of cointegrated stocks [Bonferroni correction]
* Compute the residue of the cointegrated pairs, and characterize with anARMA(p,q)
* Check if this mean reversion behaviour persists 


In [2]:
import numpy as np 
import random as rd 
import matplotlib.pyplot as plt
import seaborn as sns 
import pandas as pd    
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import coint
from sklearn.linear_model import LinearRegression

In [ ]:
__author__ = "Colleville Tanguy"
__copyright__ = "None"
__credits__ = ["None"]
__license__ = "None"
__version__ = "1.0.0"
__maintainer__ = "Colleville Tanguy"
__email__ = "tanguy.colleville@student-cs.fr"
__status__ = "Dev"

## 1. Load and split the dataset

Indice_r c'est pour dire qu'on travaille avec le log return $$\newline$$
Indice_p pour dire qu'on travaille avec le log price

In [3]:
df_r=pd.read_csv("logret_russel1000_pairtrad.csv",index_col=0)
df_p=df_r.cumsum(axis=0)##to get log price
data_p=df_p.copy()
data_r=df_r.copy()

## Passer log return à log price --> df.cumsum(axis=0)
## Passer log price à log return --> np.exp(df)-1

### Attention bien faire gaffe au NAn values le jour du test

In [4]:
print(len(df_r))

504


In [6]:
t_in,t_out=252,252

In [7]:
d_in_p,d_out_p=data_p[:t_in], data_p[t_in:]
d_in_r,d_out_r=data_r[:t_in], data_r[t_in:]

## 2. Select the subset of stocks that are I(1) in the in-sample.

In [ ]:
# en fait la dérivée d"ordre 1 est elle est faite ==> Log price c'est dérivée d'ordre 1 du log return donc si log return passe addfuller test ça fait dire que le log price est stationnaire

In [8]:
assets=d_in_p.columns

In [9]:
mask_p=[]
mask_r=[]
for asset in assets  :
    result_p = adfuller(d_in_p[asset])
    result_r = adfuller(d_in_r[asset])
    
    if result_p[1]>0.01: # SI P-value >0.01 --> ça veut dire stationnaire. Si tu taffes sur le price
        mask_p.append(False)
    if result_p[1]<0.01 : 
        mask_p.append(True)

        
    if result_r[1]<0.01:  
        mask_r.append(True)
    else : 
        mask_r.append(False) 
# Mask un peu plus opti
#sel = np.array( [i for i in range(len(d_in_p)) if (adfuller(d_in_r[i])[1]<0.01) and adfuller(d_in_p[i])[1]>0.01] )

In [10]:
mask_i_1=[]
for asset in assets  :
    result_p = adfuller(d_in_p[asset])
    result_r = adfuller(d_in_r[asset])
        
    if result_r[1]<0.01 and result_p[1]>0.01 :  
        mask_i_1.append(True)
    else : 
        mask_i_1.append(False)

In [11]:
stationnary_assets_1=assets[mask_r]
stationnary_assets_1_bis=assets[mask_i_1]
stationnary_assets_2=assets[mask_p]
print("ARE I(1) : ", stationnary_assets_1) ##all
print("ARE I(1) bis : ", stationnary_assets_1_bis) ##all
print("ARE I(2): ", stationnary_assets_2) ##all

ARE I(1) :  Index(['BPOP', 'BRK-B', 'BXP', 'BXS', 'C', 'CACI', 'CAG', 'CAH', 'CASY', 'CAT',
       'CB', 'CBSH', 'CBT', 'CCL', 'CFR', 'CHE', 'CHRW', 'CI', 'CIEN', 'CINF',
       'CKH', 'CL', 'CLI', 'CLX', 'CMA', 'COF', 'COHR', 'COO', 'COP', 'COST',
       'CPB', 'CPT', 'CR', 'CREE', 'CSCO', 'CSGP', 'CSL', 'CTAS', 'CTL', 'CVS',
       'CVX', 'CW', 'D', 'DD', 'DE', 'DGX', 'DIS', 'DISH', 'DLTR', 'DOV'],
      dtype='object')
ARE I(1) bis :  Index(['BPOP', 'BRK-B', 'BXP', 'BXS', 'C', 'CACI', 'CAG', 'CAH', 'CASY', 'CAT',
       'CB', 'CBSH', 'CBT', 'CCL', 'CFR', 'CHE', 'CHRW', 'CI', 'CIEN', 'CINF',
       'CKH', 'CL', 'CLI', 'CLX', 'CMA', 'COF', 'COHR', 'COO', 'COP', 'COST',
       'CPB', 'CPT', 'CR', 'CREE', 'CSCO', 'CSGP', 'CSL', 'CTAS', 'CTL', 'CVS',
       'CVX', 'CW', 'D', 'DD', 'DE', 'DGX', 'DIS', 'DISH', 'DLTR', 'DOV'],
      dtype='object')
ARE I(2):  Index([], dtype='object')


In [12]:
assets[mask_i_1]

Index(['BPOP', 'BRK-B', 'BXP', 'BXS', 'C', 'CACI', 'CAG', 'CAH', 'CASY', 'CAT',
       'CB', 'CBSH', 'CBT', 'CCL', 'CFR', 'CHE', 'CHRW', 'CI', 'CIEN', 'CINF',
       'CKH', 'CL', 'CLI', 'CLX', 'CMA', 'COF', 'COHR', 'COO', 'COP', 'COST',
       'CPB', 'CPT', 'CR', 'CREE', 'CSCO', 'CSGP', 'CSL', 'CTAS', 'CTL', 'CVS',
       'CVX', 'CW', 'D', 'DD', 'DE', 'DGX', 'DIS', 'DISH', 'DLTR', 'DOV'],
      dtype='object')

In [13]:
print(d_in_p.shape)
d_in_p=d_in_p[assets[mask_i_1]]
print(d_in_p.shape)

(252, 50)
(252, 50)


In [14]:
df_in_log_p_I1=df_in_log[assets[mask_i_1]]

NameError: name 'df_in_log' is not defined

In [ ]:
mask_p==mask_r

In [15]:
def diff(Y):
    return [Y[i]-Y[i-1] for i in range(1,len(Y))]

In [17]:
df_i2=d_in_p.apply(diff,axis=1)
df_i2

index
2000-01-03    [0.0026082548276860004, 0.0220099479035581, -0...
2000-01-04    [0.0005360555380855003, 0.040754200339728194, ...
2000-01-05    [0.05355796887605319, 0.03270356853161209, -0....
2000-01-06    [0.0992878412808017, 0.01028531928616339, -0.0...
2000-01-07    [0.1081179202417564, 0.037102466784682486, -0....
                                    ...                        
2000-12-22    [0.24987760138612727, 0.21828930216918996, -0....
2000-12-26    [0.2390863816018295, 0.21049270398873435, -0.6...
2000-12-27    [0.25740401753281617, 0.19411026708259588, -0....
2000-12-28    [0.2740724046112338, 0.18527276650238805, -0.6...
2000-12-29    [0.28316005769657754, 0.13590667574574278, -0....
Length: 252, dtype: object

## 3. Select the pair of cointegrated stocks [Bonferroni correction] --> sur les log PRIXXXXX

In [ ]:
matrice=np.zeros(((len(assets)),len(assets)))
N=len(assets)
for i,asset_1 in enumerate(assets):
    for j,asset_2 in enumerate(assets) :
        matrice[i,j]=coint(d_in_p[asset_1],d_in_p[asset_2])[1]# p-value de cointegration des returns 

In [29]:
import time as time 
t1=time.time()
Coint_matrice = sorted([(coint(d_in_p[i],d_in_p[j])[1],(i,j)) for i in assets[mask_i_1] for j in assets[mask_i_1] if i!=j])
print(f"Done in {time.time()-t1} s")

Done in 108.26712942123413 s


C'est une liste (p value coint, (index asset_a , index_asset_b))

In [ ]:
sns.heatmap(matrice).invert_yaxis()
# sns.clustermap(matrice)
plt.title("Map of the p-values of coint")
plt.xlabel("index of stocks")
plt.ylabel("index of stocks")
plt.show()

$ \alpha_{b}=\frac{\alpha_s}{n_{assets}}$ --> Bonferroni threshold

In [ ]:
## Bonferroni selection 
# thr_b=0.01/len(matrice.flatten())## On veut ceux qui ont des p-value supérieur à ça
thr_b=0.01/(N*N)## On veut ceux qui ont des p-value supérieur à ça
print(thr_b)

In [ ]:
mask_bonferonni=np.zeros((len(matrice),len(matrice)))
for i in range(len(matrice)):
    for j in range(len(matrice)):
        if matrice[i,j]<thr_b and i!=j:
            mask_bonferonni[i,j]=True
        else : 
            mask_bonferonni[i,j]=False

In [ ]:
sns.heatmap(mask_bonferonni).invert_yaxis()

In [ ]:
A_true=np.where(mask_bonferonni)
pairs_perso=[]
for i in range(A_true[0].shape[0]):
    print('i = {}, j = {}'.format(A_true[0][i],A_true[1][i]))
    pairs_perso.append((A_true[0][i],A_true[1][i]))

In [ ]:
pairs = list( filter(lambda x: x[0]<thr_b,Coint_matrice))## on récupère ceux qui respecte la condition de Bonferonni 

In [ ]:
print(pairs)

In [ ]:
stock_1,stock_2=pairs[0][1]
print(stock_1,stock_2)

In [ ]:
plt.figure()
plt.plot(d_in_p[stock_1].values,c="orange",label=f"{stock_1}")
plt.plot(d_in_p[stock_2].values,c="blue",label=f"{stock_2}")
plt.xlabel("time")
plt.ylabel("logprice")
plt.title("The 2 survivors to Bonferonni + I(1)")
plt.grid()
plt.legend()
plt.show()

In [ ]:
#On fait une regression sur ceux qu'on a Xi et Xj et on plot le regressé et celui qui a servi à la régression --> plot ça doit être similaire 

2 variables sont cointregrated if  : 
$ Y_t - \beta * X_t = \epsilon_t $ mais on ne connait pas $\beta$ du coup on fait une OLS : 
$ Y_t = \hat{\alpha} + \hat{\beta} X_t + \hat{u_t}$
d'où  : $\hat{u_t} = Y_t -\hat{\alpha} - \hat{\beta} X_t $
si elles le sont belles et bien alors $\hat{u_t}$ est un BB 

In [ ]:
## reg var est la valeur de prédiction une fois le modèle de régression linéaire sur stock_2 avec stock_1
rgfit = LinearRegression().fit(d_in_p[stock_1].values[np.newaxis].T,d_in_p[stock_2])## on calcul les poids de la relation linéaire de ces deux prix --> on calcul alpha chapeau et beta chapeau 
rgVar = rgfit.predict(d_in_p[stock_1].values[np.newaxis].T) ##on fait une prédiction de ce que ça donnerait pour le stock_1 on connait maintenant alpha chapeau et beta chapeau on calcul donc la valeur des alpha chap + beta chap * prix stock1

In [ ]:
plt.plot(rgVar,label=r"$\hat{\alpha} + \hat{\beta} \times stock_1$")
plt.plot(d_in_p[stock_2].values,label=f"real value of {stock_2}")
plt.tick_params(labelsize=14)
plt.xlabel('days',fontsize=16)
plt.ylabel('log-price',fontsize=16)
plt.title('in-sample',fontsize=16)
plt.grid()
plt.legend()
plt.show()

## 4. Compute the residue of the cointegrated pairs, and characterize with an ARMA(p,q)

In [ ]:
# on calcul le résidu u= priceXi- regVar ## regVar c'est Yi obtenu par regression de PriceXi
u = d_in_p[stock_2].values-rgVar # la différence entre la valeur du stock_2 et la valeur prédite à partir du stock_1

In [ ]:
# Visual representation of the regression variable
plt.plot(u,color='k')
plt.fill_between(range(252),0,np.clip(u,0,np.inf))
plt.fill_between(range(252),np.clip(u,-np.inf,0),0)
plt.xlabel('days',fontsize=16)
plt.tick_params(labelsize=14)
plt.ylabel(r'$u_t$',fontsize=16)
plt.title('in-sample',fontsize=16)
plt.show()

In [ ]:
#et on caractérise l'arma process de u 

In [ ]:
sm.graphics.tsa.plot_acf(u,lags=15)
plt.xlabel(r'$lags$',fontsize=16)
plt.ylabel('ACF',fontsize=16)
plt.tick_params(labelsize=14)
plt.grid()
plt.show()

In [ ]:
sm.graphics.tsa.plot_pacf(u,lags=15)
plt.xlabel(r'$lags$',fontsize=16)
plt.ylabel('PACF',fontsize=16)
plt.tick_params(labelsize=14)
plt.grid()
plt.show()

In [ ]:
M = {}
models_ARMA_params=[(4,0,'n'),(4,0,'nc'),
          (4,2,'n'),(4,2,'nc'),
          (5,0,'n'),(5,0,'nc'),
          (4,1,'n'),(4,1,'nc'),
          (1,1,'n'),(1,1,'nc'),
          (2,1,'n'),(2,1,'nc'),
          (1,2,'n'),(1,2,'nc'),
          (2,2,'n'),(2,2,'nc'),
          (3,3,'n'),(3,3,'nc'),
          (3,1,'n'),(3,1,'nc')] 
for p,q,tend in models_ARMA_params:
    try:
        #fit an arma (for now without trend)
        ft = ARMA(u,order=(p,q)).fit(method='mle',trend=tend)
        M[p,q,tend] = ft
    except ValueError:
        print(p,q,'Maybe not stationary')

In [ ]:
AIC = pd.DataFrame( [(m,ft.aic,ft.bic) for m,ft in M.items()],columns=['model','AIC',"BIC"] ) # colum 1 = param column2 = aic

AIC = AIC.assign(dAIC=(AIC.AIC-AIC.AIC.min()))
AIC = AIC.assign(dBIC=(AIC.BIC-AIC.BIC.min()))

AIC

In [ ]:
plt.plot(AIC["dAIC"],label="dAIC",color="blue")
plt.plot(AIC["dBIC"],label="dBIC",color="green")
# plt.plot(AIC["dAIC"].set_index("model"),label="dAIC",color="blue")
# plt.plot(AIC["dBIC"].set_index("model"),label="dBIC",color="green")
plt.grid()
plt.title("dAIC & dBIC evolution")
plt.grid()
plt.show()

In [ ]:
index_interet=[5,2]## 1,2 c'est xemple mais grace à AIC on met les indices des modèles qui nous intéresse
for i in index_interet:
    p,q,t = AIC.model[i]# p , q ,trend
    print(M[p,q,t].summary())

In [ ]:
rs = {params: M[params].resid for params in AIC.model[index_interet]}

In [ ]:
for params,r in rs.items():
    sm.graphics.tsa.plot_acf(r, lags=20)
    plt.xlabel(r'$lags$',fontsize=16)
    plt.ylabel('ACF [residue]',fontsize=16)
    plt.tick_params(labelsize=14)
    plt.title(params,fontsize=16)
    plt.show()

In [ ]:
Qtest = []  
for (p,q,trend),r in rs.items():
    pvalue = sm.stats.acorr_ljungbox(r,lags=[p+q+1],model_df=p+q,return_df=True)['lb_pvalue'][p+q+1]
    Qtest.append(( (p,q,trend), pvalue))
Qtest = pd.DataFrame( Qtest, columns=['model','residue pvalues'] )

In [ ]:
Qtest

C'est limite pour (2,1) (5,0) c'est mort 

## 5. Check if this mean reversion behaviour persists in the out-of-sample.

In [ ]:
rgVar = rgfit.predict(data_p[stock_1].values[np.newaxis].T)#on extrapole la regression à t_out mais en etant entraîné sur t_in

In [ ]:
plt.plot(rgVar)
plt.plot(data_p[stock_2].values)
plt.vlines(252,0,0.75,ls='--',color='k',label='today')
plt.ylim([0,0.5])
plt.xlabel('days',fontsize=16)
plt.tick_params(labelsize=14)
plt.ylabel(r'log-price',fontsize=16)
plt.title('in-sample + out-of-sample',fontsize=16)
plt.legend(loc='lower right',fontsize=13)

In [ ]:
## on doit regarder in-out sample Xi  e Vareg in-out et vpor so ça se suit toujours sur un plot. Sinon on fait un adfuller((Xi-regVar)[tin:])

In [ ]:
adfuller( (data_p[stock_2]-rgVar)[252:] )[1] ## on peut pas garantir que le out_sample est stationnaire car p value est trop grande ## on a fait un test de fuller sur le outsample